# First example into taichi lang

### ⚡️ When Taichi shines
- 🚀 Accelerate Python loops with GPU/CPU parallelism without rewriting everything in C++.
- 🧳 Ship the same code across Windows, Linux, macOS, NVIDIA, AMD, or plain CPUs—Taichi handles the backend juggling.
- 🧩 Model physics or custom numerical kernels that don’t fit "NumPy, but faster" libraries like CuPy or JAX.
- 😈 You lost a bet with me and now you owe Taichi a try.

**⚠️ Heads-up:** Taichi isn’t at its happiest inside notebooks; avoid re-importing it mid-session or you might need to restart the kernel. For production scripts, run them as plain Python files for rock-solid stability.

## Installation and import


In [ ]:
!pip install taichi

In [ ]:
import taichi as ti
import numpy as np
import matplotlib.pyplot as plt
import time


%matplotlib widget

### 🔌 Kick off the Taichi runtime

- `ti.init(arch=ti.gpu)` politely asks for a GPU; Taichi falls back to the fastest CPU backend if none is available.
- Kernels are JIT-compiled the first time you call them and then cached, so tweak-and-rerun cycles stay instant.
- Init happens once per interpreter session—restart the notebook if you need different precision, debugging flags, or a fresh runtime.

In [ ]:
ti.init(ti.gpu)

### 🧊 Fields = simulation memory

- Fields live inside Taichi’s runtime, not regular Python RAM, so kernels can read/write them at high speed on CPU or GPU.
- Declare them up front (`ti.field(dtype=ti.f32, shape=(nx, ny))`), then bridge to NumPy with `from_numpy` / `to_numpy` when you need to inspect or seed data.
- Because the data persists between kernel launches, fields make it easy to store simulation state, buffers, and scratch space.

In [ ]:
nx, ny = 2048, 2048
boundary_temperature = 0.0
simulation_dt = 0.05
steps_per_frame = 8
frames = 160
sim_time = 0.0

# Taichi fields that hold our state
temperature = ti.field(dtype=ti.f32, shape=(nx, ny))
temperature_next = ti.field(dtype=ti.f32, shape=(nx, ny))
conductivity = ti.field(dtype=ti.f32, shape=(nx, ny))
heat_source = ti.field(dtype=ti.f32, shape=(nx, ny))

# Dynamic heat-source descriptors
n_sources = 3
source_base = ti.Vector.field(2, dtype=ti.f32, shape=n_sources)
source_radii = ti.field(dtype=ti.f32, shape=n_sources)
source_strength = ti.field(dtype=ti.f32, shape=n_sources)
source_phase = ti.field(dtype=ti.f32, shape=n_sources)

# Regular NumPy workbench for initial conditions
x = np.linspace(0.0, 1.0, nx, dtype=np.float32)
y = np.linspace(0.0, 1.0, ny, dtype=np.float32)
X, Y = np.meshgrid(x, y, indexing='ij')

cond_init = 0.35 + 0.65 * np.sin(3 * np.pi * X) * np.sin(2 * np.pi * Y)
cond_init += 0.25 * np.exp(-((X - 0.75) ** 2 + (Y - 0.25) ** 2) / 0.01)
cond_init = np.clip(cond_init, 0.08, None).astype(np.float32)

temp_init = 0.1 * np.exp(-((X - 0.25) ** 2 + (Y - 0.75) ** 2) / 0.005)

base_centers = np.array([
    (0.32, 0.35),
    (0.68, 0.55),
    (0.5, 0.2),
], dtype=np.float32)
source_radii_data = np.array([0.002, 0.003, 0.0025], dtype=np.float32)
source_strength_data = np.array([2.0, 1.4, 1.0], dtype=np.float32)
source_phase_data = np.array([0.0, 1.2, 2.4], dtype=np.float32)


def sample_heat_field(time_point: float) -> np.ndarray:
    field = np.zeros((nx, ny), dtype=np.float32)
    for idx in range(n_sources):
        cx, cy = base_centers[idx]
        radius = source_radii_data[idx]
        strength = source_strength_data[idx]
        phase = source_phase_data[idx]
        anim_x = cx + 0.08 * np.sin(time_point * (0.8 + 0.15 * idx) + phase)
        anim_y = cy + 0.08 * np.cos(time_point * (0.6 + 0.2 * idx) + phase)
        blob = np.exp(-(((X - anim_x) ** 2 + (Y - anim_y) ** 2) / radius))
        field += strength * blob
    return field

heat_init = sample_heat_field(0.0)

# Push data to Taichi fields and parameter tables
temperature.from_numpy(temp_init.astype(np.float32))
conductivity.from_numpy(cond_init)
heat_source.from_numpy(heat_init)
source_base.from_numpy(base_centers)
source_radii.from_numpy(source_radii_data)
source_strength.from_numpy(source_strength_data)
source_phase.from_numpy(source_phase_data)

# Quick glance at the initial state
fig, axes = plt.subplots(1, 3, figsize=(11, 3.3), constrained_layout=True)
plots = [
    (temp_init.T, 'inferno', 'Initial temperature'),
    (cond_init.T, 'viridis', 'Conductivity'),
    (heat_init.T, 'magma', 'Heat source (t = 0)'),
]
for ax, (data, cmap, title) in zip(axes, plots):
    im = ax.imshow(data, origin='lower', cmap=cmap)
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

display(fig)

### ⚙️ Kernels vs. funcs

- `@ti.kernel` turns a Python function into parallel Taichi code—call it like Python, watch it launch across your device.
- `@ti.func` is an inlined helper you can reuse inside kernels for tidy math blocks without extra overhead.

### ♨️ Demo idea
We evolve the 2-D heat equation with conductivity that changes across the grid and a few spicy Gaussian heaters that orbit over time. Cool boundaries keep the whole plate from melting, so you can clearly see how the conductivity channels guide the temperature flow.

In [ ]:
inv_dx2 = 1.0  # unit grid spacing keeps the stencil tidy

@ti.func
def is_boundary(i, j):
    return (i == 0) or (j == 0) or (i == nx - 1) or (j == ny - 1)

@ti.func
def thermal_flux(i, j):
    center = temperature[i, j]
    flux = 0.0
    if i > 0:
        k_left = 0.5 * (conductivity[i, j] + conductivity[i - 1, j])
        flux += k_left * (temperature[i - 1, j] - center)
    if i < nx - 1:
        k_right = 0.5 * (conductivity[i, j] + conductivity[i + 1, j])
        flux += k_right * (temperature[i + 1, j] - center)
    if j > 0:
        k_down = 0.5 * (conductivity[i, j] + conductivity[i, j - 1])
        flux += k_down * (temperature[i, j - 1] - center)
    if j < ny - 1:
        k_up = 0.5 * (conductivity[i, j] + conductivity[i, j + 1])
        flux += k_up * (temperature[i, j + 1] - center)
    return flux * inv_dx2

@ti.func
def gaussian_blob(x, y, cx, cy, radius):
    dist2 = (x - cx) * (x - cx) + (y - cy) * (y - cy)
    return ti.exp(-dist2 / radius)

@ti.kernel
def update_heat_source(time_value: ti.f32):
    for i, j in heat_source:
        x = ti.cast(i, ti.f32) / (nx - 1)
        y = ti.cast(j, ti.f32) / (ny - 1)
        accum = 0.0
        for idx in ti.static(range(n_sources)):
            base = source_base[idx]
            radius = source_radii[idx]
            strength = source_strength[idx]
            phase = source_phase[idx]
            cx = base[0] + 0.08 * ti.sin(time_value * (0.8 + 0.15 * idx) + phase)
            cy = base[1] + 0.08 * ti.cos(time_value * (0.6 + 0.2 * idx) + phase)
            accum += strength * gaussian_blob(x, y, cx, cy, radius)
        heat_source[i, j] = accum

@ti.kernel
def diffuse_step(dt: ti.f32):
    for i, j in temperature:
        if is_boundary(i, j):
            temperature_next[i, j] = boundary_temperature
        else:
            diffusion = thermal_flux(i, j)
            temperature_next[i, j] = temperature[i, j] + dt * (diffusion + heat_source[i, j])

@ti.kernel
def swap_buffers():
    for i, j in temperature:
        temperature[i, j] = temperature_next[i, j]

### ▶️ Driving the sim
Each frame we refresh the moving heat sources, run several diffusion steps, pull the field back to NumPy, and repaint the colormap. With `ipympl` the figure updates in-place, so you can zoom around while the heat pulses drift across the plate.

In [ ]:
sim_time_local = 0.0
update_heat_source(sim_time_local)

fig = plt.figure(figsize=(5, 5))
handle = display(fig, display_id=True)


def render_frame(array, t_value):
    fig.clf()
    ax = fig.add_subplot(1, 1, 1)
    vmax = max(1.2, float(np.quantile(array, 0.995)))
    heat_map = ax.imshow(array, origin='lower', cmap='inferno', vmin=0.0, vmax=vmax)
    ax.set_title(f'Heat diffusion: t = {t_value:.2f}')
    ax.set_xticks([])
    ax.set_yticks([])
    fig.colorbar(heat_map, ax=ax, fraction=0.046, pad=0.04, label='Temperature')
    fig.canvas.toolbar_position = 'bottom'
    fig.canvas.draw()


render_frame(temperature.to_numpy().T, sim_time_local)
handle.update(fig)

for frame in range(frames):
    for _ in range(steps_per_frame):
        update_heat_source(sim_time_local)
        diffuse_step(simulation_dt)
        swap_buffers()
        sim_time_local += simulation_dt
    ti.sync()

    render_frame(temperature.to_numpy().T, sim_time_local)
    handle.update(fig)